In [0]:
from google.colab import drive
drive.mount('/movie_scripts')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /movie_scripts


In [0]:
from os import listdir
from os.path import isfile, join
mypath = '/movie_scripts/My Drive/movie_scripts'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
import random
test_set = random.choices(onlyfiles, k=len(onlyfiles) // 10)
import shutil
for test_cand in test_set:
  try:
    shutil.move(mypath + "/" + test_cand, mypath + "/test/" + test_cand)
  except:
    print("failed on ", test_cand)

failed on  shrek-the-third.html
failed on  despicable-me-2.html
failed on  affliction.html
failed on  wild-hogs.html
failed on  mrs.-brown.html


In [0]:
mypath = '/movie_scripts/My Drive/movie_scripts'

In [0]:
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
test_set = set(test_set)
for file in onlyfiles:
    if file in test_set:
        shutil.move(mypath + "/" + file, mypath + "/delete/" + file)
    else:
        shutil.move(mypath + "/" + file, mypath + "/train/" + file)

In [0]:
import pandas as pd
df = pd.read_csv(mypath + '/data_parsed_contains_word_features.csv')
trainfiles = [f for f in listdir(mypath + "/train") if isfile(join(mypath + "/train", f))]
testfiles = [f for f in listdir(mypath + "/test") if isfile(join(mypath + "/train", f))]


i = 0
for trainfile in trainfiles:
    row = df.loc[df['ScriptLink'] == trainfile].to_dict()
    if row['IsProfitable']:
        shutil.move(mypath + "/train/" + trainfile, mypath + "/train/1/" + trainfile)
    else:
        shutil.move(mypath + "/train/" + trainfile, mypath + "/train/0/" + trainfile)
        i += 1

print(i, "files are classified as 0")

for testfile in testfiles:
    row = df.loc[df['ScriptLink'] == testfile].to_dict()
    if row['IsProfitable']:
        shutil.move(mypath + "/test/" + testfile, mypath + "/test/1/" + testfile)
    else:
        shutil.move(mypath + "/test/" + testfile, mypath + "/test/0/" + testfile)
        i += 1

print(i, "files are classified as 0")

290 files are classified as 0
299 files are classified as 0


In [0]:
df

,Unnamed: 0,MovieTitle,ScriptLink,ProductionBudget,DomesticGross,WorldwideGross,Profit,IsProfitable,contains_word_features,num_words,num_chars,num_words_features,num_chars_features
0,0,avatar,avatar.html,425000000.0,760507625.0,2.789705e+09,2.364705e+09,True,"{""contains_word_<pre>"": true, ""contains_word_<...",41567,363542,"{""has_0_num_words"": 1, ""has_100_num_words"": 1,...","{""has_0_num_chars"": 1, ""has_1000_num_chars"": 1..."
1,1,star wars ep. vii: the force awakens,star-wars:-the-force-awakens.html,306000000.0,936662225.0,2.053311e+09,1.747311e+09,True,"{""contains_word_<pre></pre>"": true}",1,11,"{""has_0_num_words"": 1}","{""has_0_num_chars"": 1}"
2,2,the dark knight rises,"dark-knight-rises,-the.html",275000000.0,448139099.0,1.084439e+09,8.094391e+08,True,"{""contains_word_<pre>"": true, ""contains_word_<...",33411,302619,"{""has_0_num_words"": 1, ""has_100_num_words"": 1,...","{""has_0_num_chars"": 1, ""has_1000_num_chars"": 1..."
3,3,harry potter and the half-blood prince,harry-potter-and-the-half-blood-prince.html,250000000.0,302089278.0,9.352138e+08,6.852138e+08,True,{},0,0,{},{}
4,4,the avengers,"avengers,-the.html",225000000.0,623279547.0,1.517936e+09,1.292936e+09,True,"{""contains_word_<pre>"": true, ""contains_word_<...",26158,159957,"{""has_0_num_words"": 1, ""has_100_num_words"": 1,...","{""has_0_num_chars"": 1, ""has_1000_num_chars"": 1..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1287,1287,raymond did it,ronin.html,40000.0,3632.0,3.632000e+03,-3.636800e+04,False,"{""contains_word_<pre>"": true, ""contains_word_<...",29922,214710,"{""has_0_num_words"": 1, ""has_100_num_words"": 1,...","{""has_0_num_chars"": 1, ""has_1000_num_chars"": 1..."
1288,1288,the legend of god's gun,logan's-run.html,30000.0,243768.0,2.437680e+05,2.137680e+05,True,"{""contains_word_<pre>"": true, ""contains_word_<...",25699,160118,"{""has_0_num_words"": 1, ""has_100_num_words"": 1,...","{""has_0_num_chars"": 1, ""has_1000_num_chars"": 1..."
1289,1289,clerks,clerks.html,27000.0,3073428.0,3.894240e+06,3.867240e+06,True,"{""contains_word_<pre>"": true, ""contains_word_<...",27060,161444,"{""has_0_num_words"": 1, ""has_100_num_words"": 1,...","{""has_0_num_chars"": 1, ""has_1000_num_chars"": 1..."
1290,1290,the terrorist,eastern-promises.html,25000.0,195043.0,1.950430e+05,1.700430e+05,True,"{""contains_word_<pre>"": true, ""contains_word_<...",23615,205830,"{""has_0_num_words"": 1, ""has_100_num_words"": 1,...","{""has_0_num_chars"": 1, ""has_1000_num_chars"": 1..."


In [0]:
import ktrain
from ktrain import text

Using TensorFlow backend.


using Keras version: 2.2.4


In [0]:
(x_train, y_train), (x_test, y_test), preproc = text.texts_from_folder(mypath, maxlen=512, preprocess_mode='bert', classes=['1', '0'])

detected encoding: utf-8
downloading pretrained BERT model (uncased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: en


preprocessing test...
language: en


In [0]:
learner = ktrain.get_learner(text.text_classifier('bert', (x_train, y_train), preproc=preproc), train_data=(x_train, y_train), val_data=(x_test, y_test), batch_size=6)

Is Multi-Label? False
maxlen is 512
done.


In [0]:
learner.lr_find()

simulating training for different learning rates... this may take a few moments...
Epoch 1/1024
1099/1099 [==============================] - 8357s 8s/step - loss: 0.7417 - acc: 0.5487
Epoch 2/1024
1099/1099 [==============================] - 8351s 8s/step - loss: 0.6351 - acc: 0.6934
Epoch 3/1024
 972/1099 [=========================>....] - ETA: 15:58 - loss: 0.6095 - acc: 0.7078

In [0]:
learner.fit_onecycle(2e-5, 3)



begin training using onecycle policy with max lr of 2e-05...
Train on 1099 samples, validate on 51 samples
Epoch 1/3
1099/1099 [==============================] - 7968s 7s/step - loss: 0.6091 - acc: 0.7106 - val_loss: 0.6367 - val_acc: 0.7059
Epoch 2/3
 978/1099 [=========================>....] - ETA: 14:22 - loss: 0.5992 - acc: 0.7157

In [0]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
os.environ["CUDA_VISIBLE_DEVICES"]="0";

In [0]:
!pip install ktrain

     |████████████████████████████████| 174kB 4.9MB/s 
     |████████████████████████████████| 317kB 43.3MB/s 
     |████████████████████████████████| 112kB 42.1MB/s 
     |████████████████████████████████| 1.0MB 38.4MB/s 
     |████████████████████████████████| 245kB 40.6MB/s 
     |████████████████████████████████| 1.8MB 36.9MB/s 
  Created wheel for ktrain: filename=ktrain-0.6.1-cp36-none-any.whl size=223667 sha256=c0b0980946766000d1bb8fd5623d0339f72f1676c1c1a4aff78ee8b6c58275af
  Stored in directory: /root/.cache/pip/wheels/7b/d7/74/92a01d8592ca47c6a33ad7d9f56492a87d3d7a30ae9c186e47
  Created wheel for keras-bert: filename=keras_bert-0.80.0-cp36-none-any.whl size=37923 sha256=38dc29ab352a90a66dac0ed6fa9cdca278f04a02d9b8ca2da80bc645b9e90dd6
  Stored in directory: /root/.cache/pip/wheels/63/dc/87/3260cb91f3aa32c0f85c5375429a30c8fd988bbb48f5ee21b0
  Created wheel for seqeval: filename=seqeval-0.0.12-cp36-none-any.whl size=7424 sha256=7a65d5ee693ce30f0dc0ed6b921fd35e4f3500a737c46f9907f